In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import statsmodels.api as sm
import os
import sys

sys.path.insert(0, os.path.abspath('../developer'))

from config import MOCK_DATA, CODE, OUT
from developer.utilities import read_yaml
from developer.analysis.model import load_model
import re

In [20]:
df_total = pd.read_csv(OUT / "data" / "data_regression.csv")

##### regression

In [21]:
# Select the design matrix (explanatory variables) and the outcome variables
outcome = 'utility'
explanatory_vars = [col for col in df_total.columns if "att" in col] + ['ID']


X = df_total[explanatory_vars].astype(int)
y = df_total[outcome].astype(int)

In [22]:
X = sm.add_constant(X)
#model = sm.OLS(y, X).fit()
model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups':X['ID']})
#model = sm.M(y, X).fit(cov_type='cluster', cov_kwds={'groups': 'ID'})
#model = sm.MNLogit(y, X).fit()

In [23]:
model.summary()

c:\ProgramData\Anaconda3\envs\conjoint\Lib\site-packages\statsmodels\base\model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 26, but rank is 10
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                utility   R-squared:                       0.198
Model:                            OLS   Adj. R-squared:                  0.045
Method:                 Least Squares   F-statistic:                    0.5149
Date:                Tue, 08 Aug 2023   Prob (F-statistic):              0.845
Time:                        11:29:14   Log-Likelihood:                -137.13
No. Observations:                 132   AIC:                             318.3
Df Residuals:                     110   BIC:                             381.7
Df Model:                          21                                         
Covariance Type:              cluster                                         
==========================================================================================================
                                             coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
const                                     -0.1239      0.056     -2.228      0.026      -0.233      -0.015
att_1_PhaseOut                             0.2036      0.106      1.923      0.055      -0.004       0.411
att_1_StatusQuo                           -0.2114      0.084     -2.512      0.012      -0.376      -0.046
att_1_Stop&Maintain                       -0.2872      0.130     -2.203      0.028      -0.543      -0.032
att_1_Stop&Reduce                          0.1712      0.140      1.224      0.221      -0.103       0.445
att_2_HighInvestment&Int                  -0.1737      0.128     -1.360      0.174      -0.424       0.077
att_2_HighInvestment&Int&Consideration    -0.0190      0.121     -0.157      0.875      -0.256       0.218
att_2_LowInvestment                        0.0062      0.131      0.047      0.962      -0.251       0.264
att_2_LowInvestment&LowConsideration       0.0627      0.065      0.965      0.335      -0.065       0.190
att_3_EnergyAccess                        -0.2091      0.133     -1.572      0.116      -0.470       0.052
att_3_HealthEdu                           -0.1532      0.166     -0.921      0.357      -0.479       0.173
att_3_LowPrices                           -0.0633      0.136     -0.466      0.641      -0.329       0.203
att_3_NothingSoc                           0.0136      0.125      0.109      0.913      -0.231       0.258
att_3_Transfers                            0.2881      0.137      2.097      0.036       0.019       0.557
att_4_CreateJobs                           0.0181      0.131      0.139      0.889      -0.238       0.274
att_4_EarlyPension                         0.1730      0.124      1.394      0.163      -0.070       0.416
att_4_JobGuarantee                         0.0343      0.104      0.330      0.741      -0.169       0.238
att_4_NothingEco                          -0.0197      0.153     -0.129      0.898      -0.320       0.281
att_4_Retrain                             -0.3297      0.113     -2.924      0.003      -0.551      -0.109
att_5_CentralGov                           0.1901      0.234      0.812      0.417      -0.269       0.649
att_5_CivilNGO                             0.0874      0.225      0.388      0.698      -0.355       0.529
att_5_EnergySector                         0.0670      0.263      0.255      0.799      -0.448       0.582
att_5_LabourUnion                          0.3040      0.230      1.321      0.187      -0.147       0.755
att_5_LocalGov                            -0.2174      0.081     -2.690      0.007      -0.376      -0.059
att_5_Media                               -0.1573      0.159     -0.989      0.323      -0.469       0.155
att_5_Researchers                         -0.3977      0.099     -4.017      0.000     

In [24]:
model = load_model(OUT / "models" / "model.pickle")

In [25]:
print("Regression results for Attribute A:")
print(model.summary())

Regression results for Attribute A:
                            OLS Regression Results                            
Dep. Variable:                utility   R-squared:                       0.198
Model:                            OLS   Adj. R-squared:                  0.045
Method:                 Least Squares   F-statistic:                    0.5149
Date:                Tue, 08 Aug 2023   Prob (F-statistic):              0.845
Time:                        11:31:15   Log-Likelihood:                -137.13
No. Observations:                 132   AIC:                             318.3
Df Residuals:                     110   BIC:                             381.7
Df Model:                          21                                         
Covariance Type:              cluster                                         
                                             coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------

c:\ProgramData\Anaconda3\envs\conjoint\Lib\site-packages\statsmodels\base\model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 26, but rank is 10
  warnings.warn('covariance of constraints does not have full '


## Plots

##### 1.1 Relative support plot (Intra)

In [26]:
import plotly.graph_objects as go

# Assuming 'model' is the variable that holds your regression results
# Extract the coefficients and standard errors for each attribute level
att_1_levels = ['PhaseOut', 'Stop&Reduce', 'Stop&Maintain', 'StatusQuo']
att_2_levels = ['HighInvestment&Int', 'HighInvestment&Int&Consideration', 'LowInvestment&LowConsideration', 'LowInvestment']
att_3_levels = ['HealthEdu', 'EnergyAccess', 'LowPrices', 'Transfers', 'NothingSoc']
att_4_levels = ['CreateJobs', 'EarlyPension', 'JobGuarantee', 'Retrain', 'NothingEco']
att_5_levels = ['CivilNGO', 'EnergySector', 'LabourUnion', 'LocalGov', 'Media', 'Researchers', 'CentralGov']



#Remember to add att_6
att_levels = [att_5_levels, att_4_levels, att_3_levels, att_2_levels, att_1_levels]

att_colors = ['red', 'blue', 'green', 'orange', 'purple']  # Colors for each attribute group

fig = go.Figure()

total_levels = sum(len(levels) for levels in att_levels)

# Loop through each attribute group and add the data to the plot
for i, levels in enumerate(att_levels):
    att_coefficients = [model.params[f'att_{5-i}_{level}'] for level in levels]
    att_standard_errors = [model.bse[f'att_{5-i}_{level}'] for level in levels]

    relative_differences = [coeff - att_coefficients[-1] for coeff in att_coefficients]

    fig.add_trace(go.Scatter(
        x=relative_differences,
        y=levels,
        mode='markers',
        error_x=dict(type='data', array=att_standard_errors, color=att_colors[i], thickness=1.5),
        marker=dict(color='darkgray', size=10),
        orientation='h',
        showlegend = False,
    ))

    fig.add_shape(
        type="rect",
        x0=-1.5,  # Set a fixed value for x0, which is left side of the plot
        x1=1.5,  # Set the width of the shape to 1000 (right side of the plot)
        y0=total_levels - sum(len(l) for l in att_levels[i:]),  # Set y0 to the starting level index
        y1=total_levels - sum(len(l) for l in att_levels[i:]) + len(levels) - 1,  # Set y1 to the ending level index
        fillcolor=att_colors[i],
        opacity=0.1,  # Set the opacity for a light transparent effect
        layer="below",  # Place the rectangle below the scatter plot markers
    )

# Add a vertical line at x=0 for reference
fig.add_shape(type="line", x0=0, x1=0, y0=att_5_levels[0], y1=att_1_levels[-1], line=dict(color="gray", width=1, dash='dash'))

# Update the layout of the error bar plot
fig.update_layout(
    title='Relative Rating Differences',
    xaxis_title='',
    yaxis_title='Attribute Levels',
    yaxis=dict(categoryorder='array', categoryarray=att_5_levels),  # Set the categoryorder for y-axis based on att_1_levels
    xaxis=dict(tickformat='.2f', zeroline=False),  # Remove x-axis zeroline
    showlegend=True,  # Show legend with attribute names
    margin=dict(l=80, r=30, b=40, t=80),
    height=600,  # Set the height of the plot to 600 pixels
    width=1000,
    title_x=0.62,
)

# Show the interactive error bar plot
fig.show()


Make it into MM Marginal Means!

##### 1.2 Rating attributes with a Normalization Method.

*Normalizing the coefficients involves transforming them to a common scale, typically between 0 and 1. This makes it easier to compare the relative importance of different attributes.*


To aggregate the importance scores for different levels of the same attribute and obtain an overall importance score for each attribute, you can calculate a weighted average or sum of the importance scores of its individual levels. Here's how you can do it:

Calculate Normalized Importance Scores (NIS) for Attribute Levels:

Follow the normalization method as described earlier to calculate the normalized importance scores (NIS) for each attribute level.
Aggregate Importance Scores for Each Attribute:

a. Weighted Average Method:

Calculate the weighted average importance score for each attribute by taking the sum of the products of each level's NIS and its corresponding weight (frequency or probability of that level's occurrence in the choice sets).
This method considers both the relative importance of each level and its likelihood of being chosen in the experiment.
b. Simple Sum Method:

Sum up the normalized importance scores (NIS) of all levels within an attribute.
This method treats all levels equally in terms of their contribution to the overall importance score.
Attribute Importance Ranking:

Rank the attributes based on the aggregated importance scores. Higher scores indicate greater importance.
Example using the Weighted Average Method:

Let's consider an example with attribute "att_1" from your regression results. You have four levels: "PhaseOut," "StatusQuo," "Stop&Maintain," and "Stop&Reduce." You've already calculated the normalized importance scores (NIS) for each level as follows:

NIS(PhaseOut) = 0.8810 / 0.8810 = 1.0000
NIS(StatusQuo) = 0.2979 / 0.8810 = 0.3379
NIS(Stop&Maintain) = 0.1471 / 0.8810 = 0.1668
NIS(Stop&Reduce) = 0.9077 / 0.8810 = 1.0302
Let's assume that the frequency (or probability) of each level's occurrence in the choice sets is as follows:

PhaseOut: 30%
StatusQuo: 20%
Stop&Maintain: 25%
Stop&Reduce: 25%
Now, calculate the weighted average importance score for "att_1":
Weighted Average Importance Score for att_1 = (NIS(PhaseOut) * 0.30) + (NIS(StatusQuo) * 0.20) + (NIS(Stop&Maintain) * 0.25) + (NIS(Stop&Reduce) * 0.25)
Weighted Average Importance Score for att_1 = (1.0000 * 0.30) + (0.3379 * 0.20) + (0.1668 * 0.25) + (1.0302 * 0.25) ≈ 0.5623

Repeat this process for each attribute to obtain aggregated importance scores, and then rank the attributes based on these scores.

Remember that the choice of the weighting scheme (equal weights, frequency-based weights, or other relevant weights) depends on your specific context and research design.

##### Plot freq

In [25]:
freq = pd.read_csv(OUT / 'data' / 'data_freq.csv')

NIS

##### 1.3 General support plot for phase out

##### 1.4 Maybe: PCA Analysis:

Data Preparation:

Prepare your DCE data matrix, where each row represents a respondent's choice set, and columns represent different attribute levels.
Standardization:

Standardize the data by subtracting the mean and dividing by the standard deviation for each attribute. This ensures that all attributes are on similar scales and prevents attributes with larger variances from dominating the PCA.
Perform PCA:

Apply PCA to the standardized data matrix. The output of PCA will include the principal components and their associated eigenvalues.
Interpretation:

Examine the explained variance for each principal component. This helps you understand how much of the total variance in the data each component explains.
Look at the loadings (weights) of the original attributes on each principal component. These loadings indicate the strength and direction of the relationship between the attribute and the principal component.
Attribute Relationships:

PCA can provide insights into how attributes are related to each other. For example, attributes that have high loadings on the same principal component are positively correlated, while those with opposite loadings are negatively correlated.
Decision Support:

While PCA itself may not directly provide attribute importance scores for policy package choices, the derived principal components can help you identify patterns or relationships that might influence choices. These insights can then be used in conjunction with other methods to understand attribute importance.